In [13]:
from pathlib import Path
from utils.get_input import get_input
import numpy as np


inp = Path("./test_input").read_text()
inp = get_input(16)
inp

'#############################################################################################################################################\n#.#.........#.#...#...#...........#.....#.......#...#...........#.............#.....#.......#.......................................#......E#\n#.#.#.#####.#.#.#.#.#.#.#######.#.#.###.###.#.###.#.#.###.#######.#######.###.#.###.#.###.#.#######.###.#.#.#.#.#.#.###.###########.#####.#.#\n#.#.#.....#...#.#...#.......#...#...#.......#.................#.....#.....#.#...#...#.#...#.#.......#...#.#...#...#.#...#...#...#.........#.#\n#.#.#####.#####.#.###.#####.#.#####.###.#######.#####.###.###.#.#####.#####.#####.#####.###.#.#######.###.#.#######.#.#.#.#.#.#.#.###########\n#.#.#...#...#...#.....#...#.#.........#.#.......#.#.....#.#...#.#.....#...........#.....#.....#...#.......#...#.#.....#.#.#.#.#.#...#.......#\n#.#.###.###.#.###.#.#.#.#.#####.###.#.#.#.#######.#.#.#.###.###.#.#####.#############.#.#######.###.#.#.###.#.#.#.#####.#.#.#.#.#####.#####.#

In [14]:
inp_np = np.array([list(i) for i in inp.split("\n")])
inp_np

array([['#', '#', '#', ..., '#', '#', '#'],
       ['#', '.', '#', ..., '.', 'E', '#'],
       ['#', '.', '#', ..., '#', '.', '#'],
       ...,
       ['#', '#', '#', ..., '#', '.', '#'],
       ['#', 'S', '.', ..., '.', '.', '#'],
       ['#', '#', '#', ..., '#', '#', '#']], dtype='<U1')

In [15]:
all_dots = np.argwhere(inp_np == ".")
S = np.argwhere(inp_np == "S")
E = np.argwhere(inp_np == "E")

all_dots_s_e = np.concatenate([all_dots, S, E])
# all_dots_s_e

In [16]:
# from dijkstar import Graph, find_path
import networkx as nx


num_nodes = len(all_dots_s_e) * 4

all_dots_s_e_map = {tuple(n): i * 4 for i, n in enumerate(all_dots_s_e)}

graph = nx.Graph()

UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3

for n, i in all_dots_s_e_map.items():
    y, x = n
    # print(f"Node {i} at {y}, {x}")

    graph.add_edge(i + LEFT, i + UP, weight=1000)
    graph.add_edge(i + RIGHT, i + UP, weight=1000)
    under = (y + 1, x)
    if under in all_dots_s_e_map:
        graph.add_edge(all_dots_s_e_map[under] + UP, i + UP, weight=1)

    graph.add_edge(i + UP, i + RIGHT, weight=1000)
    graph.add_edge(i + DOWN, i + RIGHT, weight=1000)
    left_of = (y, x - 1)
    if left_of in all_dots_s_e_map:
        graph.add_edge(all_dots_s_e_map[left_of] + RIGHT, i + RIGHT, weight=1)

    graph.add_edge(i + RIGHT, i + DOWN, weight=1000)
    graph.add_edge(i + LEFT, i + DOWN, weight=1000)
    above = (y - 1, x)
    if above in all_dots_s_e_map:
        graph.add_edge(all_dots_s_e_map[above] + DOWN, i + DOWN, weight=1)

    graph.add_edge(i + DOWN, i + LEFT, weight=1000)
    graph.add_edge(i + UP, i + LEFT, weight=1000)
    right_of = (y, x + 1)
    if right_of in all_dots_s_e_map:
        graph.add_edge(all_dots_s_e_map[right_of] + LEFT, i + LEFT, weight=1)

s_i = (len(all_dots_s_e) - 2) * 4
e_i = (len(all_dots_s_e) - 1) * 4


p1 = nx.shortest_path(graph, source=s_i + RIGHT, target=e_i + UP, weight="weight")
p2 = nx.shortest_path(graph, source=s_i + RIGHT, target=e_i + RIGHT, weight="weight")
p3 = nx.shortest_path(graph, source=s_i + RIGHT, target=e_i + DOWN, weight="weight")
p4 = nx.shortest_path(graph, source=s_i + RIGHT, target=e_i + LEFT, weight="weight")


c1 = nx.path_weight(graph, p1, weight="weight")
c2 = nx.path_weight(graph, p2, weight="weight")
c3 = nx.path_weight(graph, p3, weight="weight")
c4 = nx.path_weight(graph, p4, weight="weight")

shortest = min(c1, c2, c3, c4)
print("Part 1:", shortest)

Part 1: 94444


In [17]:
# from dijkstar import Graph, find_path
import networkx as nx


num_nodes = len(all_dots_s_e) * 4

all_dots_s_e_map = {tuple(n): i * 4 for i, n in enumerate(all_dots_s_e)}

graph = nx.Graph()

UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3

for n, i in all_dots_s_e_map.items():
    y, x = n
    # print(f"Node {i} at {y}, {x}")

    graph.add_edge(i + LEFT, i + UP, weight=1000)
    graph.add_edge(i + RIGHT, i + UP, weight=1000)
    under = (y + 1, x)
    if under in all_dots_s_e_map:
        graph.add_edge(all_dots_s_e_map[under] + UP, i + UP, weight=1)

    graph.add_edge(i + UP, i + RIGHT, weight=1000)
    graph.add_edge(i + DOWN, i + RIGHT, weight=1000)
    left_of = (y, x - 1)
    if left_of in all_dots_s_e_map:
        graph.add_edge(all_dots_s_e_map[left_of] + RIGHT, i + RIGHT, weight=1)

    graph.add_edge(i + RIGHT, i + DOWN, weight=1000)
    graph.add_edge(i + LEFT, i + DOWN, weight=1000)
    above = (y - 1, x)
    if above in all_dots_s_e_map:
        graph.add_edge(all_dots_s_e_map[above] + DOWN, i + DOWN, weight=1)

    graph.add_edge(i + DOWN, i + LEFT, weight=1000)
    graph.add_edge(i + UP, i + LEFT, weight=1000)
    right_of = (y, x + 1)
    if right_of in all_dots_s_e_map:
        graph.add_edge(all_dots_s_e_map[right_of] + LEFT, i + LEFT, weight=1)

s_i = (len(all_dots_s_e) - 2) * 4
e_i = (len(all_dots_s_e) - 1) * 4


p1 = nx.shortest_path(graph, source=s_i + RIGHT, target=e_i + UP, weight="weight")
p2 = nx.shortest_path(graph, source=s_i + RIGHT, target=e_i + RIGHT, weight="weight")
p3 = nx.shortest_path(graph, source=s_i + RIGHT, target=e_i + DOWN, weight="weight")
p4 = nx.shortest_path(graph, source=s_i + RIGHT, target=e_i + LEFT, weight="weight")


c1 = nx.path_weight(graph, p1, weight="weight")
c2 = nx.path_weight(graph, p2, weight="weight")
c3 = nx.path_weight(graph, p3, weight="weight")
c4 = nx.path_weight(graph, p4, weight="weight")

shortest = min(c1, c2, c3, c4)
print("Part 1:", shortest)

all_shortest = set()

print(c1, c2, c3, c4)
from tqdm import tqdm

for c, d in tqdm(zip([c1, c2, c3, c4], [UP, RIGHT, DOWN, LEFT])):
    print("hello")
    if c == shortest:
        all_shortest_d = nx.all_shortest_paths(graph, source=s_i + RIGHT, target=e_i + d, weight="weight")
        for p in all_shortest_d:
            all_shortest.update([x // 4 for x in p])
print("Part 2:", len(all_shortest))



Part 1: 94444
95444 94444 95444 94444


0it [00:00, ?it/s]

hello
hello


1it [00:37, 37.07s/it]


KeyboardInterrupt: 

In [ ]:
all_shortest_flatten = [
    x // 4
    for xs in all_shortest
    for x in xs
]
print("Part 2:", len(set(all_shortest_flatten)))

TypeError: 'int' object is not iterable